In [1]:
import numpy as np

# Vanilla RNN Forward & Backward Pass

## Paso Forward 

In [2]:
def rnn_one_step_forward(xt, h_prev, parameters):
    Whx = parameters["Whx"]
    Whh = parameters["Whh"]
    Wyh = parameters["Wyh"]
    bh = parameters["bh"]
    by = parameters["by"]  

    # forward pass
    h_next = np.tanh(np.dot(Whx, xt) + np.dot(Whh, h_prev) + bh)
    yt_hat = softmax(np.dot(Wyh, h_next) + by)

    cache = (h_next, h_prev, xt, parameters)

    return h_next, yt_hat, cache

# New Section

In [3]:
def rnn_forward(x, h0, parameters):
  caches = []

  n_x, m, T_x = x.shape
  n_y, n_h = parameters["Wyh"].shape

  H = np.zeros((n_h, m, T_x))
  Yt_hat = np.zeros((n_y, m, T_x))

  h_next = h0

  for t in range(T_x):
    h_next, yt_hat, cache = rnn_one_step_forward(x[:,:,t], h_next, parameters)
    H[:,:,t] = h_next
    Yt_ha[:,:,t] = yt_hat
    caches.append(cache)

  caches = (caches, x)
  return H, Yt_hat, caches

## Paso Backward
**Nota:** Se está considerando el BackPropagation Through Time (BPTT) truncado.

In [4]:
def rnn_one_step_backward(dh_next, cache):
  (h_next, h_prev, xt, parameters) = cache
  Whx = parameters["Whx"]
  Whh = parameters["Whh"]
  Wyh = parameters["Wyh"]
  bh = parameters["bh"]
  by = parameters["by"]

  # gradients  
  dtanh = (1 - h_next ** 2) * dh_next 
  dWhx = np.dot(dtanh, xt.T)  
  dWhh = np.dot(dtanh, h_prev.T)
  dbh = np.sum(dtanh, axis = 1,keepdims=1)

  # inputs 
  dxt = np.dot(Whx.T, dtanh)
  dh_prev = np.dot(Whh.T, dtanh)

  gradients = {"dxt": dxt, "dh_prev": dh_prev, "dWhx": dWhx, "dWhh": dWh, "dba": dbh}

  return gradients

In [5]:
def rnn_backward(dh, caches):
  (caches, x) = caches
  (h1, h0, x1, parameters) = caches[0]

  n_h, m, T_x = dh.shape
  n_x, m = x1.shape

  # inicialización de los gradientes
  dx = np.zeros((n_x, m, T_x))
  dWhx = np.zeros((n_h, n_x))
  dWh = np.zeros((n_h, n_h))
  dbh = np.zeros((n_h, 1))
  dh0 = np.zeros((n_h, m))
  dh_prevt = np.zeros((n_h, m))    

  # Loop para cada time-step
  for t in reversed(range(T_x)):
    gradients = rnn_one_step_backward(dh[:,:,t] + dh_prevt, caches[t])

    dxt, dh_prevt = gradients["dxt"], gradients["dh_prev"], 
    dWhxt, dWhht, dbht = gradients["dWhx"], gradients["dWhh"], gradients["dbh"]
    
    dx[:, :, t] = dxt
    dWhx += dWhxt
    dWhh += dWhht
    dbh += dbht
      
  # derivada caso base
  da0 = dh_prevt
  gradients = {"dx": dx, "dh0": dh0, "dWhx": dWhx, "dWhh": dWhh,"dbh": dbh}

  return gradients

# Long Short Term Memory (LSTM) Forward & Backward Pass

La siguiente imagen muestra la arquitectura de una LSTM. 

![picture](https://drive.google.com/uc?id=11A9DOUP1F0ztCDn7-cy38S_Yc2GfHG7u)

## Paso Forward

In [6]:
def lstm_one_step_forward(xt, h_prev, c_prev, parameters):
  Wf = parameters["Wf"]
  bf = parameters["bf"]
  Wi = parameters["Wi"]
  bi = parameters["bi"]
  Wc = parameters["Wc"]
  bc = parameters["bc"]
  Wo = parameters["Wo"]
  bo = parameters["bo"]
  Wy = parameters["Wy"]
  by = parameters["by"]

  # dimensiones del xt y WY 
  n_x, m = xt.shape
  n_y, n_h = Wy.shape

  # concatenación de h_prev y xt
  concat = np.zeros((n_h + n_x, m))
  concat[: n_h, :] = h_prev
  concat[n_h :, :] = xt

  # compuerta de olvido
  ft = sigmoid(np.dot(Wf, concat) + bf)
  # compuerta de entrada
  it = sigmoid(np.dot(Wi, concat) + bi)
  # new cell memory
  cct = np.tanh(np.dot(Wc, concat) + bc)
  # la nueva memoria
  c_next = ft * c_prev + it * cct
  # compuerta de salida
  ot = sigmoid(np.dot(Wo, concat) + bo)
  # nuevo estado oculto
  h_next = ot * np.tanh(c_next)

  # el yhat predecido
  yt_hat = softmax(np.dot(Wy, h_next) + by)
  
  cache = (h_next, c_next, h_prev, c_prev, ft, it, cct, ot, xt, parameters)

  return h_next, c_next, yt_hat, cache

In [7]:
def lstm_forward(x, h0, parameters):

  caches = []

  n_x, m, T_x = x.shape
  n_y, n_h = parameters["Wy"].shape

  H = np.zeros((n_h, m, T_x))
  C = np.zeros((n_h, m, T_x))
  Y = np.zeros((n_y, m, T_x))

  h_next = h0
  c_next = np.zeros(h_next.shape)

  for t in range(T_x):
    h_next, c_next, yt, cache = lstm_cell_forward(x[:, :, t], h_next, 
                                                  c_next, parameters)
    
    H[:,:,t] = h_next
    Y[:,:,t] = yt
    C[:,:,t]  = c_next
    caches.append(cache)
      
  caches = (caches, x)

  return H, Y, C, caches

## Paso Backward

**Nota:** Se está considerando el BackPropagation Through Time (BPTT) truncado.

In [8]:
def lstm_one_step_backward(dh_next, dc_next, cache):
    
    # extrayendo valores de cache
    (h_next, c_next, h_prev, c_prev, ft, it, cct, ot, xt, parameters) = cache
        
    n_x, m = xt.shape
    n_h, m = h_next.shape

    # derivadas de las compuerta 
    # dos_t: derivada de la compuerta de salida
    # dccs_t: derivada de la new cell memory
    # dis_t: derivada de la compuerta de entrada
    # dfs_t: derivada de la compuerta de olvido

    dos_t = dh_next * np.tanh(c_next) * ot * (1 - ot)
    dccs_t = (dh_next * ot * (1 - np.tanh(c_next) ** 2) + dc_next) * it * (1 - cct ** 2)
    dis_t = (dh_next * ot * (1 - np.tanh(c_next) ** 2) + dc_next) * cct * (1 - it) * it
    dfs_t = (dh_next * ot * (1 - np.tanh(c_next) ** 2) + dc_next) * c_prev * ft * (1 - ft)
    
    # derivada de los pesos:
    # dW{f,i,c,o}: matriz de las compuertas de olvido(f), entrada(i),
    #             new cell memoria (c), salida (o)
    # db{f,i,c,o}: bias de las compuertas de olvido(f), entrada(i),
    #             new cell memoria (c), salida (o)
    dWf = np.dot(dfs_t, np.hstack([h_prev.T, xt.T]))
    dWi = np.dot(dis_t, np.hstack([h_prev.T, xt.T]))
    dWc = np.dot(dccs_t, np.hstack([h_prev.T, xt.T]))
    dWo = np.dot(dos_t, np.hstack([h_prev.T, xt.T]))
    dbf = np.sum(dfs_t, axis=1, keepdims=True)
    dbi = np.sum(dis_t, axis=1, keepdims=True)
    dbc = np.sum(dccs_t, axis=1, keepdims=True)
    dbo = np.sum(dos_t, axis=1, keepdims=True)

    dh_prev = np.dot(Wf[:, :n_h].T, dfs_t) + np.dot(Wc[:, :n_h].T, dccs_t)
    dh_prev += np.dot(Wi[:, :n_h].T, dis_t) + np.dot(Wo[:, :n_h].T, dos_t)

    dc_prev = (dh_next * ot * (1 - np.tanh(c_next) ** 2) + dc_next) * ft
    dxt = np.dot(Wf[:, n_h:].T, dfs_t) + np.dot(Wc[:, n_h:].T, dccs_t)
    dxt += np.dot(Wi[:, n_h:].T, dis_t) + np.dot(Wo[:, n_h:].T, dos_t)
    
    gradients = {"dxt": dxt, "dh_prev": dh_prev, "dc_prev": dc_prev, 
                 "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
                 "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

    return gradients

In [9]:
def lstm_backward(dh, caches):

  # extrayendo valores de la cache
  (caches, x) = caches
  (h1, c1, h0, c0, f1, i1, cc1, o1, x1, parameters) = caches[0]

  n_h, m, T_x = dh.shape
  n_x, m = x1.shape

  # inicializando los parametros
  dx = np.zeros((n_x, m, T_x))
  dh0 = np.zeros((n_h, m))
  dh_prevt = np.zeros((n_h, m))
  dc_prevt = np.zeros((n_h, m))
  dWf = np.zeros((n_h, n_h + n_x))
  dWi = np.zeros((n_h, n_h + n_x))
  dWc = np.zeros((n_h, n_h + n_x))
  dWo = np.zeros((n_h, n_h + n_x))
  dbf = np.zeros((n_h, 1))
  dbi = np.zeros((n_h, 1))
  dbc = np.zeros((n_h, 1))
  dbo = np.zeros((n_h, 1))

  # loop a travez del time-step
  for t in reversed(range(T_x)):
    gradients = lstm_cell_backward(dh[:,:,t] + dh_prevt, dc_prevt, caches[t])

    dx[:,:,t] = gradients["dxt"]
    dWf += gradients["dWf"]
    dWi += gradients["dWi"]
    dWc += gradients["dWc"]
    dWo += gradients["dWo"]
    dbf += gradients["dbf"]
    dbi += gradients["dbi"]
    dbc += gradients["dbc"]
    dbo += gradients["dbo"]

  dh0 = gradients["dh_prev"]
      
  gradients = {"dx": dx, "dh0": dh0, "dWf": dWf,"dbf": dbf, "dWi": dWi,"dbi": dbi,
              "dWc": dWc,"dbc": dbc, "dWo": dWo,"dbo": dbo}

  return gradients

# Gated Recurrent Units (GRU)

En cada sección debes de codificar las funciones correspondientes al paso forward y backward, tal como se hizo para Vanilla RNN y LSTM. Se te dará un esqueleto base para codificar, si gusta modificalas a tu manera. En caso de encontrar un error, **esto de existir serán adrede ;)** , sientete libre de modificar las funciones. Considera que el y predecido corresponde a un problema de multi-clasificación 

Se muestra una imagen de la arquitectura de una GRU. Imagen credito de CS224n: Natural Language Processing with Deep Learning, Chris Manning.
![picture](https://drive.google.com/uc?id=1A2i7TH_RXJITXNixkO1UUlibOXHYqw5f)

## Paso Forward (20 puntos)



In [2]:
def gru_one_step_forward(xt, h_prev, parameters):
  # parametros
  # ==> CODIFICA AQUI

  # dimensiones del xt y WY 
  # ==> CODIFICA AQUI

  # concatenación de h_prev y xt
  # ==> CODIFICA AQUI

  # COMPUERTAS
  # ==> CODIFICA AQUI

  # el yhat predecido
  # ==> CODIFICA AQUI
  
  # almacenamienot para el paso backward
  # ==> codifica aqui

  return h_next, c_next, yt_hat, cache

In [ ]:
def gru_forward(x, h0, parameters):

  caches = []

  n_x, m, T_x = x.shape
  n_y, n_h = parameters["Wy"].shape

  H = np.zeros((n_h, m, T_x))
  C = np.zeros((n_h, m, T_x))
  Y = np.zeros((n_y, m, T_x))

  h_next = h0
  c_next = np.zeros(h_next.shape)

  for t in range(T_x):
    # ==> codifica aqui
      
  caches = (caches, x)

  return H, Y, caches

## Paso Backward (30 puntos)

**Nota:** Puedes usar BackPropagation Through Time (BPTT) truncado. Pero hay puntos extras si realizas el BPTT normal. Te recomiendo que antes de codificar hagas las derivaciones a mano, tal como se hizo en clase para vanilla RNN y LSTM.

In [3]:
def gru_one_step_backward(dh_next, cache):
    
    # extrayendo valores de cache
    # guiate de la misma función del LSTM y RNN
    # ===> CODEA AQUI
        
    n_x, m = xt.shape
    n_h, m = h_next.shape

    # derivadas de las compuerta 
    # considera cuantas compuertas tiene el GRU, y mira la diferencia con las
    # LSTM
    # ===> CODEA AQUI
    
    # derivada de los pesos:
    # dW{f,i,c,o}: matriz de las compuertas de ___, ___, ___
    # db{f,i,c,o}: bias de las compuertas de ___, ___, ___
    # ===> CODEA AQUI

    # derivada de h_prev, 
    # ===> CODEA AQUI

    # derivada de dxt
    # ===> CODEA AQUI
    
    # almanacena las derivadas calculadas en un diccionario "gradients"
    # ===> CODEA AQUI

    return gradients

In [5]:
def gru_backward(dh, caches):

  # extrayendo valores de la cache
  (caches, x) = caches
  # ===> CODEA AQUI

  n_h, m, T_x = dh.shape
  n_x, m = x1.shape

  # inicializando los parametros
  # ===> CODEA AQUI

  # loop a travez del time-step
  for t in reversed(range(T_x)):
    # ===> CODEA AQUI

  dh0 = gradients["dh_prev"]
      
  # almanacena las derivadas calculadas en un diccionario "gradients"
  # ===> CODEA AQUI     

  return gradients

## Derivaciones (50 puntos)

Realiza las derivaciones a mano para el BPTT para la GRU, tal como se hizo en clase. Se muestra un ejemplo parcial para LSTM. Recuerda no solo expresar cada derivada usando la regla de la cadena, si no, también realiza la derivada de la función tanh, etc.

![picture](https://drive.google.com/uc?id=1YdZsdX6TGjsFqy96WqIrh47_QZPrLu3g)

Este ejemplo muestra las derivadas de las compuertas, pero no de la matriz de peso y el bias. Tu tienes que hallar de todo.
